In [36]:
# Convert ts file to json

import re

txt = ""
with open("learnsets.ts") as file:
    txt = file.read()

txt = re.split("=", txt)[1]
txt = re.sub("//.*", "", txt)
txt = re.sub("(\w+):", "\"\g<1>\":", txt)
txt = re.sub(",(\s*)(]|})", "\g<1>\g<2>", txt)
txt = re.sub(";", "", txt)

with open("learnsets.json", "w") as file:
    file.write(txt)
